# Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
theta = "\u03B8"
Theta = "\u0398"

# Load Data

In [ ]:
## Zero Delay ##

time_zero = 37.958  # from BiSe

## HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results ##

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
# Load original dataset
fp = r"E:\atully\arpes_data\2023_February\6eV\TR"
fn = "TR3_avg_g_kw.h5"  # 2.15 eV center energy; -1 to 2 ps
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

## Fix time axis on k-corrected TR scan ##

file = "TR3_Ali_avg.h5"  # 2.15 eV center energy; -1 to 2 ps
_, _, phi_or_time, _ = loading_functions.load_hdf5(ddir, file)
phi = phi_or_time

In [ ]:
# Load FFT, k-corrected Dataset

ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR3_avg_g_kw_filteredFFT_0.00int.h5"]  # 2.15 eV center energy; -1 to 2 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad3_fft = ARPES_DATA[files[0]]

In [ ]:
# Load original dataset
fp = r"E:\atully\arpes_data\2023_February\6eV\TR"
fn = "TR4_avg_g_kw.h5"  # 2.15 eV center energy; -1 to 2 ps
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

## Fix time axis on k-corrected TR scan ##

file = "TR4_Ali_avg.h5"  # 2.15 eV center energy; -1 to 2 ps
_, _, phi_or_time, _ = loading_functions.load_hdf5(ddir, file)
phi = phi_or_time

In [ ]:
# Load FFT, k-corrected Dataset

ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = [
    "TR4_avg_g_kw_filteredFFT_0.00int.h5"
]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad4_fft = ARPES_DATA[files[0]]

In [ ]:
## Set Default Titles ##

# yaxis_title = "E - E<sub>HOMO_Top</sub> (eV)"
yaxis_title = "E - E<sub>HOMO</sub> (eV)"
# xaxis_title = Theta
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"

# Analysis

In [ ]:
## Set up general parameters ##

## This integrates from zero delay to 1 ps
slice_center = 0.5
integration = 1

slice_val = time_zero + tr_functions.ps_to_mm(slice_center, time_zero)
int_range = tr_functions.ps_to_mm(integration)  # TODO: make this able to be a tuple...


# ## This integrates from -0.5 to 0.5 ps
# slice_center = 0
# integration = 1

# slice_val = time_zero + tr_functions.ps_to_mm(slice_center, time_zero)
# int_range = tr_functions.ps_to_mm(integration)  # TODO: make this able to be a tuple...


## Slicing in time to look for angular dispersion
slice_dim = "z"

# xlim = (-12, 12)  # theta
xlim = (-0.15, 0.15)  # k-corrected
ylim = None
x_bin = 2
y_bin = 2

In [ ]:
all_vals = []
for ad in [ad3_fft, ad4_fft]:
    # for ad in [ad3_fft]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )
x3_fft, y3_fft, d3_fft = all_vals[0]
x4_fft, y4_fft, d4_fft = all_vals[1]

In [ ]:
# x3_fft, y3_fft, d3_fft = tr_functions.slice_datacube(
#     ad_dataclass=ad3_fft,
#     slice_dim=slice_dim,
#     slice_val=slice_val,
#     int_range=int_range,
#     xlim=xlim,
#     ylim=(
#         ad3_fft.energy[57],
#         ad3_fft.energy[1007],
#     ),  # get rid of zero padding on datasets
#     x_bin=x_bin,
#     y_bin=y_bin,
#     norm_data=True,
#     plot_data=False,
# )

In [ ]:
## Adjust energy axis to be relative to HOMO ##

homo_zero = False
homo_zero = True

if homo_zero:
    y4_fft = (
        y4_fft - homo_400
    )  # homo is negative, so energy scale will increase, because we're referencing a negative number rather than zero
    y3_fft = y3_fft - homo_700

In [ ]:
## Optional Limit Dataset ##

xlim = xlim
# ylim = (2.05, np.max(y3_fft))  # theta
ylim = (2.05, 2.55)  # k-corrected, TR3
ylim = (2.0, 2.55)

## TR3 ##
x3fft, y3fft, d3fft = analysis_functions.limit_dataset(
    x3_fft, y3_fft, d3_fft, xlim=xlim, ylim=ylim
)

d3fft = analysis_functions.norm_data(d3fft)

## TR4 ##
ylim = (2.45, np.max(y4_fft))  # k-corrected, TR4
ylim = (2.402, np.max(y4_fft))  # k-corrected, TR4

x4fft, y4fft, d4fft = analysis_functions.limit_dataset(
    x4_fft, y4_fft, d4_fft, xlim=xlim, ylim=ylim
)

d4fft = analysis_functions.norm_data(d4fft)
# d4fft[np.where(d4fft > 0.5)] = 0.5  # limit dataset to cmax=0.5 for denoising purposes

In [ ]:
## Stitch and Average Datasets ##

theta2, energy2, data2 = x3fft, y3fft, d3fft * 2  # CT1
theta1, energy1, data1 = x4fft, y4fft, d4fft  # CT2

xlim = None
ylim = None

# xlim = (-0.16, 0.16)
# # xlim = (-0.17, 0.17)
# ylim1 = (2.00, 2.5)  # 2.07, 2.5
# ylim2 = (2.48, 3.15)

x1, y1, dataslice1 = analysis_functions.limit_dataset(
    theta1,
    energy1,
    data1,
    xlim=xlim,
    ylim=ylim,
)

x2, y2, dataslice2 = analysis_functions.limit_dataset(
    theta2,
    energy2,
    data2,
    xlim=xlim,
    ylim=ylim,
)

xs, ys, ds = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

In [ ]:
## Plot data ##

# ## homo top!!
# y4_new = y4fft - 0.3
# y3_new = y3fft - 0.3

fig = tr_functions.thesis_fig(
    # title=f"CT<sub>1</sub> Angular Dispersion",
    title=f"CT<sub>2</sub> Angular Dispersion",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    # width=500,
    width=600,
    # dtick_y=0.2,
)

# ## TR 3
# fig.add_trace(go.Heatmap(x=x3fft, y=y3fft, z=d3fft, coloraxis="coloraxis"))

# fig.update_coloraxes(cmin=0, cmax=0.13)
# fig.update_coloraxes(cmin=0, cmax=0.9)  # when dataset is limited


## TR 4
# x_new, y_new, d_new = analysis_functions.limit_dataset(
#     x4_fft,
#     y4_fft,
#     d4_fft,
#     xlim=None,
#     ylim=None,
# )

# fig.add_trace(
#     go.Heatmap(
#         x=x_new,
#         y=y_new,
#         z=d_new,
#         coloraxis="coloraxis",
#     )
# )

# fig.update_coloraxes(cmin=0.0, cmax=0.25)

fig.add_trace(
    go.Heatmap(
        x=x4fft,
        # y=y4_new,
        y=y4fft,
        z=d4fft,
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0.0, cmax=0.25)


## Stitched
# fig.add_trace(go.Heatmap(x=xs, y=ys, z=ds, coloraxis="coloraxis"))

# fig.update_coloraxes(cmin=0.0, cmax=0.25)  # full energy range (CT1 + CT2)


fig.update_coloraxes(colorscale="Blues", reversescale=False)


fig.show()

In [ ]:
## Get and Plot 1D Data --> EDC ##
# ylim = (2.1, 2.8)
ylim = (2.0, np.max(ys))
# ylim = (2.0, 2.44)
# ylim = (2.45, np.max(ys))

xlim = None
ylim = None

x_plot, y_plot, data_plot = xs, ys, ds
x_plot, y_plot, data_plot = x4fft, y4fft, d4fft


fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title="Intensity (arb. u)",
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    # height=1000,
    # width=600,
    height=800,
    width=300,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_plot, y=y_plot, data=data_plot, ylims=ylim, x_range=xlim
)

# fig.add_trace(go.Scatter(x=y_1d, y=col, line=dict(color=colors[0])))
fig.add_trace(go.Scatter(x=col, y=y_1d, line=dict(color=colors[0])))

# fig.update_yaxes(range=(2.0, np.max(ys)))
fig.update_yaxes(range=(np.min(y4fft), np.max(y4fft)))

fig.show()

In [ ]:
x = y_1d
data = col

offset_type = "constant"
plot_fit = True


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Gaussians
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=2.1, sigma=0.1)
# gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=2.1, sigma=0.1)

gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=2.55, sigma=0.1)
gauss3 = fitting_functions.make_gaussian(num="C_", amplitude=1, center=2.8, sigma=0.1)
gauss4 = fitting_functions.make_gaussian(num="D_", amplitude=1, center=2.9, sigma=0.1)


## Full model
full_model = lm.models.Model(fermi_dirac) + gauss1 + gauss2 + offset

# full_model = lm.models.Model(fermi_dirac) + gauss1 + offset

full_model = gauss2 + gauss3 + gauss4 + offset


params = full_model.make_params()

# params["center"].value = 2.0325  # Based on TR data
# # params["center"].vary = False
# T = 10.6  # measurement temp
# k_B = 8.617333e-5  # eV/K

# ## initial params
# params["theta"].value = k_B * (10.6)
# params["theta"].min = 0
# params["amp"].value = 1
# params["amp"].min = 0

# # ## final params
# params["theta"].value = 0.02027560
# # params["theta"].vary = False
# params["amp"].value = 173.920569
# # params["amp"].vary = False
# params["c"].value = 132.268557
# # params["c"].vary = False

# params["iA__center"].min = 1.6
# params["iA__center"].max = 2.0
# params["iB__center"].min = 2.0
# params["iB__center"].max = 2.2

# params["iA__center"].min = 2.0
# params["iA__center"].max = 2.3
params["iB__center"].min = 2.3
params["iB__center"].max = 2.8
params["iC__center"].min = 2.6
params["iC__center"].max = 3.0
params["iD__center"].min = 2.8
params["iD__center"].max = 3.0

# params["iA__fwhm"].max = 0.5
# params["iA__fwhm"].min = 0.1
# params["iB__fwhm"].max = 0.5
# params["iC__fwhm"].max = 0.5
# params["iD__fwhm"].max = 0.5


fit = full_model.fit(data, x=x, params=params)
if plot_fit:
    fit.plot()

# print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
# print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} eV")
print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} eV")
print(f"Center C = {fit.params['iC__center'].value:.2f} eV")
print(f"FWHM C = {fit.params['iC__fwhm'].value:.3f} eV")
print(f"Center D = {fit.params['iD__center'].value:.2f} eV")
print(f"FWHM D = {fit.params['iD__fwhm'].value:.3f} eV")

In [ ]:
fit_ct2 = fit

In [ ]:
# fit_ct1 = fit

In [ ]:
# fit.result

In [ ]:
## Plot Data and Fit
fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title="Intensity (arb. u)",
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    height=800,
    # width=315,
    dtick_y=0.2,
    # height=700,
    width=500,
)

fig.add_trace(go.Scatter(x=col, y=y_1d, line=dict(color=colors[0]), name="data"))

# fit = fit_ct2
# fig.add_trace(
#     go.Scatter(
#         x=fit.eval(x=y_1d[np.where(y_1d > 2.45)]),
#         y=y_1d[np.where(y_1d > 2.45)],
#         name="fit CT2",
#     )
# )

fit = fit_ct2
fig.add_trace(
    go.Scatter(
        x=fit.eval(x=y_1d),
        y=y_1d,
        name="fit CT2",
    )
)

components = fit.eval_components(x=y_1d)
for model_name, model_value in list(components.items())[0:3]:
    fig.add_annotation(
        x=fit.eval(x=fit.params[f"{model_name}center"].value),
        y=fit.params[f"{model_name}center"].value,
        text=f'{fit.params[f"{model_name}center"].value:.2f} ',
        font=dict(size=24, color=colors[0]),
        ax=50,
        ay=30,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

# fit = fit_ct1
# fig.add_trace(
#     go.Scatter(
#         x=fit.eval(x=y_1d[np.where(y_1d < 2.45)]),
#         y=y_1d[np.where(y_1d < 2.45)],
#         name="fit CT1",
#     )
# )
# components = fit.eval_components(x=y_1d)
# for model_name, model_value in list(components.items())[1:2]:
#     fig.add_annotation(
#         x=fit.eval(x=fit.params[f"{model_name}center"].value),
#         y=fit.params[f"{model_name}center"].value,
#         text=f'{fit.params[f"{model_name}center"].value:.2f} ',
#         font=dict(size=18, color=colors[0]),
#         ax=50,
#         ay=30,
#         bgcolor="white",
#         # opacity=1,
#         # bordercolor=colors[0],
#         # borderwidth=2,
#         # borderpad=4,
#     )

# fig.update_yaxes(range=(2.0, np.max(ys)))
fig.update_yaxes(range=(np.min(y4fft), np.max(y4fft)))

fig.show()

# Denoised Data

## Generate denoised datasets

In [ ]:
# x_dn, y_dn, d_dn = x_array, y_array, data_array

# ## Save to .itx (igor) file
# cnn.save_to_igor_itx(
#     "test.itx", [x_dn], [d_dn], ["trarpes"], [y_dn]
# )  # takes every other y value to make dataset smaller
# cnn.fix_itx_format("test.itx")  # fix itx formatting for denoising website

In [ ]:
# x_dn, y_dn, d_dn = x3fft, y3fft, d3fft
# # x_dn, y_dn, d_dn = x4_fft, y4_fft, d4fft

# # Save to .itx (igor) file
# cnn.save_to_igor_itx(
#     "test.itx", [x_dn], [d_dn], ["trarpes"], [y_dn]
# )  # take every other y value to make dataset smaller
# cnn.fix_itx_format("test.itx")  # fix itx formatting for denoising website

## Load denoised data

In [ ]:
# Denoised Data
# fp_dn = r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\CNN\CT1 HS dispersion"
# fn_dn = "CT1_ang_disp_dn.itx"
# fn_dn = "CT1_ang_disp_limY_dn.itx"

# fp_dn = (
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\CNN\CT2 HS dispersion\0to1ps"
# )
# fn_dn = "CT2_ang_disp_dn.itx"
# fn_dn = "CT2_ang_disp_cmax05_dn.itx"
# fp_dn = r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\CNN\CT2 HS dispersion\-0.5to0.5ps"
# fn_dn = "CT2_ang_disp_zerodelay_dn.itx"


fp_dn = r"C:\Users\atully\OneDrive\Physics.UBC\PhD\2DGC\Figs"
# fn_dn = "CT1_ang_disp_dn.itx"
fn_dn = "CT2_ang_disp_dn.itx"

# title = f"CT<sub>1</sub> Angular Dispersion"
title = f"CT<sub>2</sub> Angular Dispersion"
yaxis_title = "E - E<sub>HOMO</sub> (eV)"
# xaxis_title = Theta
xaxis_title = xaxis_title

x, y, data_dn = loading_functions.load_denoised_data(fp_dn, fn_dn)

## Analyze deniosed data

In [ ]:
## homo top!!
# y_new = y - 0.3

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data_dn), coloraxis="coloraxis")
)

# fig.add_trace(go.Heatmap(x=x, y=y, z=data_dn, coloraxis="coloraxis"))

# fig.update_coloraxes(cmin=0, cmax=0.13)

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    # title=f"EDC of CT<sub>1</sub>",
    title=f"EDC of CT<sub>2</sub>",
    yaxis_title=yaxis_title,
    xaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=300,
    dtick_y=0.1,
    dtick_x=0.4,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x, y=y, data=analysis_functions.norm_data(data_dn), ylims=None, x_range=None
)

# Plot Data
fig.add_trace(go.Scatter(x=col, y=y_1d, name="data", line=dict(color=colors[0])))